In [1]:
import pandas as pd
recvae = pd.read_csv("submission/predict_RecVAE.csv", header=None)
als = pd.read_csv("submission/predict_ALS.csv", header=None)
lightgcn = pd.read_csv("submission/predict_LightGCN.csv", header=None)
sar = pd.read_csv("submission/predict_SAR.csv", header=None)
xsimgcl = pd.read_csv("submission/predict_XSimGCL.csv", header=None)
cl4rec = pd.read_csv("submission/predict_CL4Rec.csv", header=None)

/tmp/ipykernel_23404/464803147.py:2: DtypeWarning: Columns (965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000) have mixed types. Specify dtype option on import or set low_memory=False.
  recvae = pd.read_csv("submission/predict_RecVAE.csv", header=None)


In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

def load_and_process_recommendations(file_paths):
    """
    Đọc và xử lý các file khuyến nghị từ nhiều mô hình
    """
    model_predictions = {}
    for model_name, file_path in file_paths.items():
        df = pd.read_csv(file_path, header=None)[[0,1,2,3]]
        # Chuyển đổi DataFrame thành dictionary với key là user_id và value là list các khuyến nghị
        predictions = {str(row[0]): list(row[1:]) for _, row in df.iterrows()}
        model_predictions[model_name] = predictions
    return model_predictions

def calculate_jaccard_similarity(set1, set2):
    """
    Tính toán độ tương đồng Jaccard giữa hai tập hợp
    """
    intersection = len(set(set1) & set(set2))
    union = len(set(set1) | set(set2))
    return intersection / union if union != 0 else 0

def analyze_model_agreement(model_predictions):
    """
    Phân tích độ đồng thuận giữa các mô hình
    """
    user_similarities = defaultdict(list)
    model_names = list(model_predictions.keys())
    
    # Với mỗi người dùng, tính toán độ tương đồng giữa các cặp mô hình
    for user_id in model_predictions[model_names[0]].keys():
        similarities = []
        # So sánh từng cặp mô hình
        for i in range(len(model_names)):
            for j in range(i+1, len(model_names)):
                model1, model2 = model_names[i], model_names[j]
                recs1 = model_predictions[model1][user_id]
                recs2 = model_predictions[model2][user_id]
                similarity = calculate_jaccard_similarity(recs1, recs2)
                similarities.append(similarity)
        
        # Tính trung bình độ tương đồng cho người dùng này
        avg_similarity = np.mean(similarities)
        user_similarities[user_id] = avg_similarity
    
    return user_similarities

def get_extreme_cases(user_similarities, threshold_high=0.5, threshold_low=0.25):
    """
    Lấy ra các trường hợp có độ đồng thuận cao và thấp
    """
    high_agreement = {k: v for k, v in user_similarities.items() if v >= threshold_high}
    low_agreement = {k: v for k, v in user_similarities.items() if v <= threshold_low}
    
    return high_agreement, low_agreement

def analyze_recommendations():
    # Định nghĩa đường dẫn đến các file
    file_paths = {
        'RecVAE': "submission/predict_RecVAE.csv",
        'ALS': "submission/predict_ALS.csv",
        'LightGCN': "submission/predict_LightGCN.csv",
        'SAR': "submission/predict_SAR.csv",
        'XSimGCL': "submission/predict_XSimGCL.csv"
    }
    
    # Đọc và xử lý dữ liệu
    model_predictions = load_and_process_recommendations(file_paths)
    
    # Phân tích độ đồng thuận
    user_similarities = analyze_model_agreement(model_predictions)
    
    # Lấy ra các trường hợp đặc biệt
    high_agreement, low_agreement = get_extreme_cases(user_similarities)
    
    # In kết quả phân tích
    print(f"Tổng số người dùng: {len(user_similarities)}")
    print(f"Số người dùng có độ đồng thuận cao: {len(high_agreement)}")
    print(f"Số người dùng có độ đồng thuận thấp: {len(low_agreement)}")
    
    # In ra một vài ví dụ
    print("\nVí dụ về người dùng có độ đồng thuận cao:")
    for user_id, similarity in list(high_agreement.items())[:5]:
        print(f"User {user_id}: {similarity:.3f}")
        
    print("\nVí dụ về người dùng có độ đồng thuận thấp:")
    for user_id, similarity in list(low_agreement.items())[:5]:
        print(f"User {user_id}: {similarity:.3f}")
        
    return high_agreement, low_agreement, user_similarities

# Chạy phân tích
high_agreement, low_agreement, user_similarities = analyze_recommendations()

/tmp/ipykernel_23404/774754908.py:11: DtypeWarning: Columns (965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None)[[0,1,2,3]]


Tổng số người dùng: 2817
Số người dùng có độ đồng thuận cao: 427
Số người dùng có độ đồng thuận thấp: 1156

Ví dụ về người dùng có độ đồng thuận cao:
User NcVPv81tzp: 0.600
User F1PQozhklt: 0.600
User bs2VMaU4pt: 0.530
User Ea1H5XClBM: 0.600
User XdXjiurCzj: 0.520

Ví dụ về người dùng có độ đồng thuận thấp:
User ljMBWT9UXW: 0.000
User WZXCQtOWJt: 0.180
User xlB2rvpQKu: 0.060
User ih5TXNF0Wb: 0.240
User 4btbKDKvSl: 0.130


In [5]:
user_id = low_agreement.keys()

In [6]:
recvae = pd.read_csv("submission/predict_RecVAE.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
als = pd.read_csv("submission/predict_ALS.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
lightgcn = pd.read_csv("submission/predict_LightGCN.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
sar = pd.read_csv("submission/predict_SAR.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
xsimgcl = pd.read_csv("submission/predict_XSimGCL.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
cl4rec = pd.read_csv("submission/predict_CL4Rec.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])

/tmp/ipykernel_23404/3414659978.py:1: DtypeWarning: Columns (965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000) have mixed types. Specify dtype option on import or set low_memory=False.
  recvae = pd.read_csv("submission/predict_RecVAE.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])


In [35]:
xsimgcl = pd.read_csv("submission/predict_XSimGCL_falsedata.csv", names = ['user_id'] + [f'item_id_{i}' for i in range(1,1001)])

FileNotFoundError: [Errno 2] No such file or directory: 'submission/predict_XSimGCL_falsedata.csv'

In [18]:
for i in range(1,1001):
    xsimgcl.loc[~xsimgcl.user_id.isin(user_id), f'item_id_{i}'] = "0"

In [19]:
xsimgcl.to_csv('test_submit/predict.csv', index = None, header = None)

In [20]:
data = pd.read_csv('data/training_set.csv')
data.head()

,UserId,ItemId,Click,Purchase
0,tyviMi4b8Q,tyviMi4b8Q,1,0
1,tyviMi4b8Q,t3fB9Nq1VY,1,0
2,tyviMi4b8Q,X7ZdDwPBKM,1,0
3,tyviMi4b8Q,r4bieaZn66,1,0
4,tyviMi4b8Q,cRl2I3cVGB,1,0


In [30]:
item_react = data[data.UserId.isin(low_agreement.keys())]['ItemId'].unique()

In [34]:
data[data.ItemId.isin(item_react)]

,UserId,ItemId,Click,Purchase
3,tyviMi4b8Q,r4bieaZn66,1,0
5,tyviMi4b8Q,0N6lG60HQq,1,0
8,tyviMi4b8Q,NOxmzBDWzA,1,0
9,tyviMi4b8Q,lvSijpy1MJ,1,0
10,tyviMi4b8Q,qrgmRqBJAa,1,0
...,...,...,...,...
389914,Uo2X1ah0bf,hRapAMLApl,1,0
389915,Uo2X1ah0bf,6S3euGhLbW,1,0
389917,Uo2X1ah0bf,M1M2Wkp92o,1,0
389920,16jkTS9Vj2,ZBmAO23TI8,1,0


In [29]:
data[data.ItemId.isin(item_react)].groupby('ItemId')['Click'].sum().describe()

count    19783.000000
mean        10.994440
std         15.276635
min          1.000000
25%          3.000000
50%          6.000000
75%         13.000000
max        318.000000
Name: Click, dtype: float64